In [1]:
import pickle as pkl
import numpy as np
import pandas as pd

from zipfile import ZipFile
from sklearn.metrics import accuracy_score, matthews_corrcoef, confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.backend import clear_session

from twitter_nlp_toolkit.tweet_sentiment_classifier import tweet_sentiment_classifier
from twitter_nlp_toolkit.file_fetcher import file_fetcher



In [2]:
clear_session()

In [3]:
chunk = 1 # Fraction of data to train on - you can reduce for debugging for speed
model_path = '.models'
redownload=True

Here we download pre-trained models and a validation dataset. The models have been trained on the Sentiment140 dataset, taken form here: https://www.kaggle.com/kazanova/sentiment140

The validation data is hand-labeled airline customer feedback taken from https://www.figure-eight.com/data-for-everyone/

In [4]:
if redownload: 
    
    # Validation data
    file_fetcher.download_file('https://www.dropbox.com/s/muqov03s9vtbe92/tweets_airline.zip?dl=1',"tweets_airline.zip")
    


tweets_airline.zip: 100%|#########################################################| 1.07M/1.07M [00:00<00:00, 4.40MB/s]


In [5]:
# Load the validation data

test_data = pd.read_csv('tweets_airline.zip', header=0, names=['Index', 'Sentiment', 'Sentiment_confidence',
                                                                'Negative_reason', 'Negative_reason_confidence',
                                                                'Airline', 'Airline_sentiment_gold', 'Handle',
                                                                'Negative_reason_gold', 'Retweet_count', 'Text',
                                                                'Tweet_coord', 'Time', 'Location', 'Timezone'])

In [6]:
# Remove the unlabeled test data

test_data['Labels'] = (test_data['Sentiment'] == 'positive') * 2
test_data['Labels'] = test_data['Labels'] + (test_data['Sentiment'] == 'neutral') * 1
test_data['Labels'] = test_data['Labels'] / 2

test_data.set_index('Labels')
test_data = test_data[test_data.Labels != 0.5]

In [7]:
# Download the small ensemble
# Executing this cell starts a 500MB download

Classifier = tweet_sentiment_classifier.SentimentAnalyzer()
Classifier.load_small_ensemble()

bow_1Mtweets
Default parameters file not found or not working...
Trying .models/small_ensemble/bow_1Mtweets
Loading BoW model bow_1Mtweets from legacy parameter file
BoW model bow_1Mtweets loaded successfully
glove_100D_10N_340000V_719685T_Twitter_0
Default parameters file not found or not working...
Trying .models/small_ensemble/glove_100D_10N_340000V_719685T_Twitter_0
Loading GloVE model glove_100D_10N_340000V_719685T_Twitter_0 from legacy parameter file
Pre-trained embedding model glove_100D_10N_340000V_719685T_Twitter_0 loaded successfully
glove_200D_10N_20000V_343332T_Twitter_7
Default parameters file not found or not working...
Trying .models/small_ensemble/glove_200D_10N_20000V_343332T_Twitter_7
Loading GloVE model glove_200D_10N_20000V_343332T_Twitter_7 from legacy parameter file
Pre-trained embedding model glove_200D_10N_20000V_343332T_Twitter_7 loaded successfully


We santiy check the models: 

In [8]:
Classifier.predict(['I am happy', 'I am sad', 'I am cheerful', 'I am mad'])


array([1., 0., 1., 0.])

We test the model on an airline customer feedback dataset.

In [9]:
# Executing this cell takes several minuites on a laptop

predictions = Classifier.predict(test_data['Text'])


In [10]:
print('Test Accuracy:  {:.3f}'.format(accuracy_score(test_data['Labels'], predictions)))
print('Test MCC:  {:.3f}'.format(matthews_corrcoef(test_data['Labels'], predictions)))
confusion_matrix(test_data['Labels'], predictions)

Test Accuracy:  0.812
Test MCC:  0.570


array([[7305, 1873],
       [ 297, 2066]], dtype=int64)

We have accuracy of just over 80%.

We split our evaluation dataset into validation and testing and check for poor-performing models:

In [11]:
valX, testX, valY, testY = train_test_split(test_data['Text'], test_data['Labels'], test_size=0.5, stratify=test_data['Labels'])

In [12]:
# Executing this cell takes several minuites on a laptop

Classifier.trim_models(valX, valY, threshold=0.7)

Model bow_1Mtweets score: 0.808
Model glove_100D_10N_340000V_719685T_Twitter_0 score: 0.796
Model glove_200D_10N_20000V_343332T_Twitter_7 score: 0.803


All three models perform well, so none have been pruned

In [13]:
predictions = Classifier.predict(testX)

print('Test Accuracy:  {:.3f}'.format(accuracy_score(testY, predictions)))
print('Test MCC:  {:.3f}'.format(matthews_corrcoef(testY, predictions)))
confusion_matrix(testY, predictions)

Test Accuracy:  0.805
Test MCC:  0.556


array([[3623,  966],
       [ 160, 1022]], dtype=int64)

To improve our accuracy, we can refine the models on our airline data. The early stopping procedure (enabled by default to use 20% of the training data for validation) should minimize overfitting.

In [14]:
Classifier.refine(valX, valY)

 Preprocessed 5770 tweets 

 Filtered data
 Preprocessed 5770 tweets 
Filtered data
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 4616 samples, validate on 1154 samples
Epoch 1/500
4616/4616 [==============================] - 4s 817us/sample - loss: 0.4980 - acc: 0.7790 - val_loss: 0.4483 - val_acc: 0.8059
Epoch 2/500
4616/4616 [==============================] - 0s 27us/sample - loss: 0.4816 - acc: 0.7933 - val_loss: 0.4360 - val_acc: 0.8198
Epoch 3/500
4616/4616 [==============================] - 0s 27us/sample - loss: 0.4656 - acc: 0.8000 - val_loss: 0.4244 - val_acc: 0.8276
Epoch 4/500
4616/4616 [==============================] - 0s 26us/sample - loss: 0.4635 - acc: 0.8059 - val_loss: 0.4134 - val_acc: 0.8388
Epoch 5/500
4616/4616 [==============================] - 0s 26us/sample - loss: 0.4458 - acc: 0.8174 - val_loss: 0.4030 - val_acc: 0.8518
Epoch 6/500
4616/4616 [==============================] - 0s 25us/sample - loss: 0.4310 - acc: 0.8338 - val_loss: 0.3932 - val_a

Epoch 58/500
4616/4616 [==============================] - 0s 29us/sample - loss: 0.3157 - acc: 0.8882 - val_loss: 0.2861 - val_acc: 0.8969
Epoch 59/500
4616/4616 [==============================] - 0s 29us/sample - loss: 0.3037 - acc: 0.8867 - val_loss: 0.2860 - val_acc: 0.8960
Epoch 60/500
4616/4616 [==============================] - 0s 28us/sample - loss: 0.3130 - acc: 0.8910 - val_loss: 0.2859 - val_acc: 0.8943
Epoch 61/500
4616/4616 [==============================] - 0s 29us/sample - loss: 0.3174 - acc: 0.8869 - val_loss: 0.2858 - val_acc: 0.8943
Epoch 62/500
4616/4616 [==============================] - 0s 28us/sample - loss: 0.3092 - acc: 0.8897 - val_loss: 0.2858 - val_acc: 0.8960
Epoch 63/500
4616/4616 [==============================] - 0s 28us/sample - loss: 0.2994 - acc: 0.8897 - val_loss: 0.2857 - val_acc: 0.8960
Epoch 64/500
4616/4616 [==============================] - 0s 29us/sample - loss: 0.3077 - acc: 0.8845 - val_loss: 0.2856 - val_acc: 0.8960
Epoch 65/500
4616/4616 [===

Epoch 117/500
4616/4616 [==============================] - 0s 26us/sample - loss: 0.2856 - acc: 0.8964 - val_loss: 0.2679 - val_acc: 0.9003
Epoch 118/500
4616/4616 [==============================] - 0s 26us/sample - loss: 0.2867 - acc: 0.8947 - val_loss: 0.2680 - val_acc: 0.9003
Epoch 119/500
4616/4616 [==============================] - 0s 25us/sample - loss: 0.2831 - acc: 0.8967 - val_loss: 0.2683 - val_acc: 0.9012
Epoch 120/500
4616/4616 [==============================] - 0s 25us/sample - loss: 0.2743 - acc: 0.8973 - val_loss: 0.2745 - val_acc: 0.9012
Epoch 121/500
4616/4616 [==============================] - 0s 25us/sample - loss: 0.2874 - acc: 0.8964 - val_loss: 0.2743 - val_acc: 0.9012
Epoch 122/500
4616/4616 [==============================] - 0s 26us/sample - loss: 0.2801 - acc: 0.9014 - val_loss: 0.2740 - val_acc: 0.9012
Epoch 123/500
4616/4616 [==============================] - 0s 25us/sample - loss: 0.2869 - acc: 0.8964 - val_loss: 0.2737 - val_acc: 0.9012
Epoch 124/500
4616/4

Epoch 7/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.3970 - acc: 0.8421 - val_loss: 0.3526 - val_acc: 0.8605
Epoch 8/500
4616/4616 [==============================] - 0s 33us/sample - loss: 0.3906 - acc: 0.8484 - val_loss: 0.3428 - val_acc: 0.8709
Epoch 9/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.3815 - acc: 0.8551 - val_loss: 0.3341 - val_acc: 0.8735
Epoch 10/500
4616/4616 [==============================] - 0s 33us/sample - loss: 0.3632 - acc: 0.8616 - val_loss: 0.3262 - val_acc: 0.8830
Epoch 11/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.3584 - acc: 0.8631 - val_loss: 0.3189 - val_acc: 0.8830
Epoch 12/500
4616/4616 [==============================] - 0s 33us/sample - loss: 0.3524 - acc: 0.8718 - val_loss: 0.3123 - val_acc: 0.8821
Epoch 13/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.3459 - acc: 0.8683 - val_loss: 0.3064 - val_acc: 0.8813
Epoch 14/500
4616/4616 [======

Epoch 66/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.2803 - acc: 0.8995 - val_loss: 0.2702 - val_acc: 0.9038
Epoch 67/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.2818 - acc: 0.8990 - val_loss: 0.2698 - val_acc: 0.9029
Epoch 68/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.2852 - acc: 0.8984 - val_loss: 0.2695 - val_acc: 0.9021
Epoch 69/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.2761 - acc: 0.8956 - val_loss: 0.2692 - val_acc: 0.9021
Epoch 70/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.2713 - acc: 0.8988 - val_loss: 0.2689 - val_acc: 0.9021
Epoch 71/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.2800 - acc: 0.8982 - val_loss: 0.2686 - val_acc: 0.9012
Epoch 72/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.2723 - acc: 0.9029 - val_loss: 0.2682 - val_acc: 0.9003
Epoch 73/500
4616/4616 [===

Epoch 125/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.2630 - acc: 0.9027 - val_loss: 0.2637 - val_acc: 0.9081
Epoch 126/500
4616/4616 [==============================] - 0s 33us/sample - loss: 0.2712 - acc: 0.9021 - val_loss: 0.2635 - val_acc: 0.9090
Epoch 127/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.2617 - acc: 0.9027 - val_loss: 0.2635 - val_acc: 0.9081
Epoch 128/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.2522 - acc: 0.9066 - val_loss: 0.2637 - val_acc: 0.9064
Epoch 129/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.2461 - acc: 0.9058 - val_loss: 0.2639 - val_acc: 0.9038
Epoch 130/500
4616/4616 [==============================] - 0s 33us/sample - loss: 0.2628 - acc: 0.9073 - val_loss: 0.2642 - val_acc: 0.9038
Epoch 131/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.2516 - acc: 0.9051 - val_loss: 0.2645 - val_acc: 0.9047
Epoch 132/500
4616/4

Epoch 184/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.2459 - acc: 0.9112 - val_loss: 0.2601 - val_acc: 0.9064
Epoch 185/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.2399 - acc: 0.9075 - val_loss: 0.2600 - val_acc: 0.9064
Epoch 186/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.2405 - acc: 0.9099 - val_loss: 0.2599 - val_acc: 0.9064
Epoch 187/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.2411 - acc: 0.9075 - val_loss: 0.2599 - val_acc: 0.9064
Epoch 188/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.2460 - acc: 0.9075 - val_loss: 0.2598 - val_acc: 0.9064
Epoch 189/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.2394 - acc: 0.9077 - val_loss: 0.2598 - val_acc: 0.9073
Epoch 190/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.2375 - acc: 0.9058 - val_loss: 0.2597 - val_acc: 0.9073
Epoch 191/500
4616/4

In [15]:
test_predictions = Classifier.predict(testX)

print('Test Accuracy:  {:.3f}'.format(accuracy_score(testY, test_predictions)))
print('Test MCC:  {:.3f}'.format(matthews_corrcoef(testY, test_predictions)))
confusion_matrix(testY, predictions)

Test Accuracy:  0.914
Test MCC:  0.721


array([[3623,  966],
       [ 160, 1022]], dtype=int64)

Now we have accuracies of over 90%! 

In [16]:
Classifier.evaluate(testX, testY)

Model bow_1Mtweets accuracy_score: 0.891
Model glove_100D_10N_340000V_719685T_Twitter_0 accuracy_score: 0.907
Model glove_200D_10N_20000V_343332T_Twitter_7 accuracy_score: 0.915


{'bow_1Mtweets': 0.8906601975394213,
 'glove_100D_10N_340000V_719685T_Twitter_0': 0.9071218159764339,
 'glove_200D_10N_20000V_343332T_Twitter_7': 0.9147461445156818,
 'ensembled': 0.9143995841275342}